### Popular Big Data Fromats

* Data format is an important aspect of working with big data

* The recurring topic is "There ain't such a thing as free lunch"

```"There ain't no such thing as a free lunch", also known as "there is no such thing as a free lunch" (TINSTAAFL), is an expression that describes the cost of decision-making. The expression conveys the idea that things appearing free always have some cost paid by somebody.``` **https://www.investopedia.com/terms/t/tanstaafl.asp**


### Key Considerations for Big Data Formats

* In light of the TINSTAAFL principle, the following challenges are always posed when choosing a data format for big data projects:
* **Compression**: Different file formats offer varying levels of compressibility with specific algorithms.
* **Splittability**: Evaluate how easily a file format can be split.
  * As highlighted when we discussed the embarrassingly parallel paradigm, the capacity to split a file for distributed processing across multiple machines can be of critical importance.
* **Column vs. Row Storage**: The importance of individual columns (or variables) in your dataset may vary.
* **Data Types and Schema Evolution**:  Consider whether it's necessary to enforce data types.
  * Given the scale of big data (think petabytes), it's unrealistic to assume files can be easily regenerated whenever schema changes occur.

### File Format: an Intuition

* In big data, selecting an appropriate storage format is crucial for optimizing performance, conserving storage space.
* The right choice can lead to time savings, reduced complexity, and lower costs.
* Most of us are familiar with row-based formats -- think MS Excel, where each row represents a table entry.

| Transaction Date     | Nb Items     | Total       |
|------------------    |----------    |---------    |
| 01/01/2001           | 4            | 1852.14     |
| 01/01/2001           | 3            | 968.00      |
| ...             | ...     | ...     |

### File Format: an Intuition - Cont'd
    
* The current format may not be suitable for certain data types or operations.

* E.g.: sales information contains an extremely large volume of daily transactions—say hundreds of thousands.
    * Transaction dates would be redundantly repeated many times, wasting storage space.
    * A dictionary-like format, where the key is the transaction date, might be more storage-efficient.

```python
{"01/01/2001": ((4, 1852.14), (3,  968.00), ...), "01/02/2001":(...), ... }
```

* This format would also make day-based calculations more efficient.
  * For instance, calculating the number of transactions or total sales for each day.


### File Format: an Intuition - Cont'd

* If the objective is to calculate the total sales:
  * In a table format, sifting through millions of lines would be required to determine a single total sales figure.
  * Using dates as keys, you'd simply index into each specific date to tally up the total sales.
* Another option is to utilize a row-based data format.
  * Here, reading a single line could be enough to compute the total sales.

|              |      |  |   | |
| :---              |    :----:  | :--------: | :--------: |:---:|
| **Totals**             | 1852.14    | 968.00   | 256.21 | `...` |
| **Transaction Dates** | 01/01/2001 | 01/01/2001 | 01/02/2001 | `...` |
| **Nb Items**               | 4             | 3       |  2  | `...` |

### File Formats Decisions

* Four key factors should be considered when choosing file formats:
    1. Row vs Column
      * What types of analytics will be performed?
    2. Schema Management
      * Is it likely that my data schemas will change over time?
    3. Splittability
      * Is it possible to distribute data across multiple files or even servers?
    4. Compression
      * How well does the format compress?


### 1. Row- and Column-Based Formats

* This is a crucial factor to consider when choosing a format for big data.

![](https://www.dropbox.com/s/an5fg7xl2uvnfb8/row_col_format.png?dl=1)

### Issue with In Memory data Layout

* What's the issue when data is not stored consecutively in RAM?
* Example: Storing rows but needing to extract a single column.
* This makes data retrieval inefficient.
* Buzzword Alert: It's a "Stride" problem!
  * A "stride" is the gap between data points we're interested in.
  * In row based data, to extract columns, we need to skip n−1 columns to get to the next data point in the column.
  * This is called "strided access"

### Strided Access

* Consider the following dat
* 
```python
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
```

* A typical, "non-strided" access pattern might read every element in the array consecutively from start to finish, like so:
```python
1 -> 2 -> 3 -> 4 -> 5 -> 6 -> 7 -> 8 -> 9 -> 10
```

* In a strided access pattern with a stride of 2, you would skip every other element:
```python
1 -> 3 -> 5 -> 7 -> 9
```

### Strided Access - cont'd

* you're trying to read down a single column, you'd be skipping $n−1$ elements to get to the next one in that column
  * Strided access pattern with a stride of $n$
 
* Strided access is generally less efficient than consecutive access due to the way CPU caching mechanisms and memory hierarchies work.
  *  Less effective use if the cache
  *  Potentially more time-consuming operations to retrieve the required data.
  *  Modern CPUs love data that's close together!
* This is a "data locality" problem.
   * More time spent retrieving data

### Caching and Performance

* Caches store frequently accessed data.
* Cache Miss occurs when the data isn't in the cache, it takes longer to fetch.
* Strided Access often results in more cache misses.
  * This slows down the application or system.
* The issue is covered at lenght  in Concurrent and High-Performance Programming (ICS432),  Operating Systems (ICS332),  Machine-Level and Systems Programming (ICS312)

### 1. Row- and Column-Based Formats

* Row-based: Best suited when you need to access all the data.
  * For instance, when constructing a machine learning model that uses all features and instances.
    * You can sidestep loading the entire dataset into RAM by reading it in chunks (batches).
    * This approach necessitates frequent conditional access to multiple columns.
* Column-based storage: Ideal for colun-specific tasks.
  * Examples include calculating total sales or even aggregating data by date, among others.


### Row-Based Formats

* Commonly employed in a range of mainstream applications, from web log files to structured database systems like MySQL and Oracle.
* To process the entire dataset, one would need to read each line sequentially.
* This format is a go-to choice for Online Transactional Processing (OLTP).
  * OLTP systems typically handle CRUD operations (Create, Read, Update, and Delete) at the individual record level.
  * A key focus for OLTP systems is ensuring data integrity in environments where multiple users are accessing the data.
    * Performance in OLTP systems is generally assessed by the number of transactions executed per second.
      * We'll delve deeper into this topic when discussing big data platforms.

### Column-Based Formats

* Data is organized by columns.
* Streamlines computation on selected columns.
  * For example, you can easily compute the mean or standard deviation for a particular column.
    * Why do you think this operation is more efficient in a column-based format?

* Highly conducive to compression.
  * Compression algorithms like GZIP and pkzip perform better when handling sequences of similar data types.

  ```python
  [1,2,3,...], 
  ["John", "Janet", "Michael", ...]
  ```

This is more efficient to compress than:
```python
[[1, "John", "Doe", "125,000"], [2, "Janet", "Smith", "195,129"], ...]
````

### Column-Based Formats

* Disk and network are often the bottlenecks in large, distributed systems.
* Employing compression minimizes read IO and data transfers, making your analysis faster.

* Column-based data storage is commonly known as OLAP (Online Analytical Processing).
  * OLAP is a computing approach designed to quickly answer multi-dimensional queries. 
  * More on OLAP when we dicuss platforms.

<div align="center">
<img src="https://www.dropbox.com/scl/fi/oqongt8biqxwzrsesl889/olap_dimensions.png?rlkey=qjlqqcby2da2qa4v1b9c7ot31&dl=1" width=1100/> 
</div>



### Compression of Row vs. Columnar Data

* ** Data Homogeneity** : This means that the same compression algorithm can be applied more effectively.
  * Example: A column with only integers can be efficiently compressed using an algorithm tailored for integers.
* **Reduced Cardinality**: Columns often have fewer unique values.
  * Example: Run-length encoding can be extremely effective in such cases.
* **Range of Values**:  Columns often have a narrow range of values
  * Example: Delta encoding could be highly effective here.
  * initial data: `[100, 105, 110, 101, 100]` >encode> `100 [5, 10, 1, 0]`
  * Think about `diff` in code for example.

* **Pedictive models**: use predictive compresison when "good enough" is "good enough"


In [5]:
import random
random.choices([1,2,3,4], k=6)

[3, 3, 4, 1, 2, 3]

In [6]:
import random
random.choices("ACGT", k=6)

['T', 'T', 'G', 'C', 'C', 'T']

In [7]:
import string 

print(string.printable)
print(string.digits)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

0123456789


In [11]:
import zlib 
import string

# let's randomly generate two string of 1000, an ASCII and an INT

random_ASCII = random.choices(string.printable, k=10_000)
random_numbers = random.choices(string.digits, k=10_000)
print(len(zlib.compress( str.encode("".join(random_ASCII)))))
print(len(zlib.compress( str.encode("".join(random_numbers)))))

8414
5081


In [28]:
import numpy
ratios = []
for i in range(10):
    random_ASCII = random.choices(string.printable, k=10_000)
    random_numbers = random.choices(string.digits, k=10_000)
    len_ascii = len(zlib.compress( str.encode("".join(random_ASCII)))) 
    len_numbers = len(zlib.compress( str.encode("".join(random_numbers))))
    ratios.append(len_ascii/len_numbers)
    
numpy.mean(ratios)

1.6547445239154617

In [19]:
string.ascii_uppercase

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [29]:
ratios = []
for i in range(10):
    random_ASCII = random.choices(string.printable, k=10_000)
    random_uppercase = random.choices(string.ascii_uppercase, k=10_000)
    len_ascii = len(zlib.compress( str.encode("".join(random_ASCII)))) 
    len_uppercase = len(zlib.compress( str.encode("".join(random_uppercase))))
    ratios.append(len_ascii/len_uppercase)    
numpy.mean(ratios)

1.3462144408226877

### Examples of OLAP versus OLTP in Amazon

![](https://www.dropbox.com/s/cxhwtc5s582tnp2/amazon_olap_oltp.png?dl=1)

### Column-based formats: Advantages and Disadvantages

<u>Advantages</u>:
* Columnar storage of data can sometimes yield 100x-1000x performance improvements, particularly for wide datasets


<u>Disadvantages</u>:
  *  Not efficient with CRUD operations
  * Difficult to access all features of a single instance
    * Need to parse all columns to read items at position $i$
  * Hard to read by a human
  * Can be more CPU intensive to write for very large data.



### 2- Datatype and Schema Enforcement and Evolution

* "Schema" in a database context, means the structure and organization of the data  
    * Structure: datatypes, missing values, primary keys, etc, indices, etc.
    * Organization: relationships across tables.

* Here, we mainly refer to the data type
* In text format, (e.g.: table with values separated by space), datatype cannot be declared or enforced

* Declaring the type of a value provides some advantages.
  * Storage requirements: String categories will require more storage than boolean (2 bytes)
  * Data validity: Verifies the dataset is valid and prevents entry errors (e.g., age = Johnn)
  * Compression: there are good strategies for compressing different data types 

### 2- Datatype & Schema Enforcement and Evolution - Cont'd


* In the event that there is no guarantee that data won't change in the future, you may need to consider schema evolution.


* When evaluating schema evolution, there are a few key questions to ask of any data format:
  * How easy is it to update a schema (such as adding a field, removing or renaming a field)?
  * How will different versions of the schema impact applications?
  * How fast can the schema be processed?


### 3- Splitability

* Big data such as monthly logs, yearly transactions, daily airplane sensors recordings, can often comprise many millions of records.

* Often useful to split the data across multiple machines and execute each computation separately

* Some file formats are more amenable to splitting than others.

### 3- Splitability - Row-Based

Row-based formats can be split along row boundaries

```
# file 1 with n lines
01/01/2001           4            1852.14
01/01/2001           3            968.00
...
```

* Splitting can be done
  * Randomly plitting `file 1` with `n` observations across `m` total machines is easy.

    * Each machine gets `ceiling(n/m)` unique lines, last machine gets remaining lines

 * Splitting based on one or more fields: 
    * Partitioning a rown-based file over particular column values can be difficult if data is stored in a random order.
    * May require sorting the data first

### 3- Splitability: Row-Based, Nested 

* Larg column-based data can be more difficult to split

``` 
file 2
{"01/01/20014": [(4, 1852.14), (3, 968.00)], ....}
```

* You cannot easily split this file this file format without parsing the file first.
  * Need to read the compelte file to split it into chunks.
    * Data may need to ne loaded in RAM first.


### 3- Splitability: Column-Based, Nested


* A column-based format can be split if the comutation is column-specific.

```
# file 3
date: 01/01/2001, 01/01/2001
nb_items: 4, 3
totals: 1852.14, 968.00
```

Splitting can only e done column-wise:
* In the example above, each machine is concerned with a computation on a specific variable. For example:
  * Machine 1 takes `date` data and computes the number of sales per month
  * Machine 2 takes the `nb_items` data and computes the total number of sales
  * Machine 3 takes the `totals` data and computes the total sales values
 
* Machines don't have any knowledge of variables that are not given.
  * E.g., if machine three is not given date info and cannot compute, for example, the monthly or weekly sales average.


### 4- Compression


* When working on a distributed system, data transfer can be a serious bottleneck
* Compression can substantially improve runtime and storage requirements

* We illustrated "naively" that columnar data can achieve better compression rates than row-based data
  * Simple way to think about it: column will have a lot more duplicate values:
      * Ex. Age Column: 21, 22, 21, 24, 25, 21, 22, 21, 19, 21, 21, 22, ....
      
* Note that complex compression algorithms on very large files can save on space but substantially increase compute time.
    * Uncompression/re-compression needs to occur every time you need to access the data.


### Standardization and File Formats

* One can always choose their own format for the file
  * Many companies may choose to do so internally for many reasons.
  * E.g.:

```
FIRST_NAME_1\sLAST_NAME_1\tFIRST_NAME_2\sLAST_NAME_2\tFIRST_NAME_3\sLAST_NAME_3...
JOBTITLE_1\sSALARY_1\tJOBTITLE_2\sSALARY_2\tJOBTITLE_3\sSALARY_3
```

* However, there are many benefits to using a standard file format. E.g.:
  * Clarity and productivity: eliminating the need for guesswork or extra searching for answer. Plus there is no need to maintain internal documentation, which makes it easier to get answers online when issues arise.

  * Quality: standard formats are designed by large teams and used extensively, which provides opportunities to optimize them

  * Interoperability: your data is no longer locked to your company (or compartmentalized) and can be used across platforms.

* Some of the most used formats are CSV, JSON, Parquet, AVRO, HDF5
  * All very well supported in Python


### CSV File Format

* Files in the CSV (Comma-separated values) format are usually used to exchange tabular data
  * Plain-text file (readable characters)
 
* CSV is a row-based file format: each row of the file is a separate data instance
  * May or may not contain a header
* Structure is conveyed through explicit commas
  * Text commas are encapsulated in double quotes

```
Title,Author,Genre,Height,Publisher
"Computer Vision, A Modern Approach","Forsyth, David",data_science,255,Pearson
Data Mining Handbook,"Nisbet, Robert",data_science,242,Apress
Making Software,"Oram, Andy",computer_science,232,O'Reilly
...
```

### CSV File Format

* CSV format is not fully standardized
  * Other characters can be used to separate files, such as tabs (tsv) or spaces (ssv)
 
* Data relationships across multiple CSV files are not expressed in the file format
  * Use same column names to indicate "foreign key" relationship
 

* Native support in Python
```python
import csv
csv.reader(csvfile, delimiter=',', quotechar='"')
# use csv ...
```

In [1]:
# All_Time_Worldwide_Box_Office_partial.csv
import csv
with open('data/All_Time_Worldwide_Box_Office.csv')  as csvfile:
    movies_file = csv.DictReader(csvfile, delimiter=',', quotechar='"')
    i = 0 
    for line in movies_file:
        print(f"Line {i}: {line}")
        i+=1
        if i ==10:
            break

Line 0: {'Rank': '1', 'Year': '2009', 'Movie': 'Avatar', 'WorldwideBox Office': '$2,845,899,541', 'DomesticBox Office': '$760,507,625', 'InternationalBox Office': '$2,085,391,916'}
Line 1: {'Rank': '2', 'Year': '2019', 'Movie': 'Avengers: Endgame', 'WorldwideBox Office': '$2,797,800,564', 'DomesticBox Office': '$858,373,000', 'InternationalBox Office': '$1,939,427,564'}
Line 2: {'Rank': '3', 'Year': '1997', 'Movie': 'Titanic', 'WorldwideBox Office': '$2,207,986,545', 'DomesticBox Office': '$659,363,944', 'InternationalBox Office': '$1,548,622,601'}
Line 3: {'Rank': '4', 'Year': '2015', 'Movie': 'Star Wars Ep. VII: The Force Awakens', 'WorldwideBox Office': '$2,064,615,817', 'DomesticBox Office': '$936,662,225', 'InternationalBox Office': '$1,127,953,592'}
Line 4: {'Rank': '5', 'Year': '2018', 'Movie': 'Avengers: Infinity War', 'WorldwideBox Office': '$2,044,540,523', 'DomesticBox Office': '$678,815,482', 'InternationalBox Office': '$1,365,725,041'}
Line 5: {'Rank': '6', 'Year': '2015',

### CSV Pros and Cons
<u>Pros:</u>
* Human-readable and easy to edit manually
* Provides a simple scheme
* Can be processed by almost all existing applications
* Easy to implement and parse;
* Compact (compared to, for instance JSON or MXL)
* Column headers are written only once

<u>Cons:</u>
* No guarantees about data integrity, i.e., data won't be missing or won't be in a different type than expected.
* Adding complex structures to a data structure is not possible
  * May need to reference other files to implement nesting
* There is no standard way to present binary data
* Lack of a universal standard can cause 

### JSON File Format

* JSON (JavaScript Object Notation)
* Open standard file format that uses human-readable text
  * FIle typically stored using `.json` extension
* Became popular as a space-saving alternative to Extensible Markup Language (XML)
* Inspired by JavaScript objects but is a language-independent data format
* Very similar to the combination of Python's lists and dicts
* Also supported natively in Python
  
```python
import json
json.load(...)
```

* The defacto language of the web
  * Supported in all modern languages and particularly web languages.

### JSON File Structure

* JSON supports the following types.

* Scalar values
    * `Numbers`: e.g. 3
    * `String`: Sequence of Unicode characters surrounded by double quotation marks.
    * `Boolean`: `true` or `false`.

* Collections:
    * `Array`: A list of values surrounded by square brackets `[]`
    * `Dictionaries`: key" value pairs separated by a comma(,) and enclosed in `{}`
      * Keys are strings and values can be any valid scalar or collection
* [See the following for more details](https://docs.fileformat.com/web/json/)
* [See the following very good (useful) validator for validating JSON files or records](https://jsonformatter.curiousconcept.com/#)

In [40]:
my_data = [ 
    {'First Name': "John", "Occupation": "Student", "Salary": 120_000, "volunteer": False}, 
    {'First Name': "John", "Occupation": "Student", "salary": None, "volunteer": True}
]
my_data


[{'First Name': 'John',
  'Occupation': 'Student',
  'Salary': 120000,
  'volunteer': False},
 {'First Name': 'John',
  'Occupation': 'Student',
  'salary': None,
  'volunteer': True}]

```python
json.load
json.loads
json.dump
json.dumps
```

In [49]:
import json
json_representation = json.dumps(my_data)
print(json_representation)
# Note the changes between the Python dict and the JSON string

[{"First Name": "John", "Occupation": "Student", "Salary": 120000, "volunteer": false}, {"First Name": "John", "Occupation": "Student", "salary": null, "volunteer": true}]


### Working with the Python `json` library


* `All_Time_Worldwide_Box_Office_partial.json`  structure
```json
[
 {
  "Rank": "1",
  "Year": "2009",
  "Movie": "Avatar",
  "WorldwideBox Office": "$2,845,899,541",
  "DomesticBox Office": "$760,507,625",
  "InternationalBox Office": "$2,085,391,916"
 },
 {
  "Rank": "2",
  "Year": "2019",
  "Movie": "Avengers: Endgame",
  "WorldwideBox Office": "$2,797,800,564",
  "DomesticBox Office": "$858,373,000",
  "InternationalBox Office": "$1,939,427,564"
 },
 ...
]
```

In [3]:
import json
json_file = open('data/All_Time_Worldwide_Box_Office_partial.json') 
movies_data = json.load(json_file)
movies_data[0:3]


[{'Rank': '1',
  'Year': '2009',
  'Movie': 'Avatar',
  'WorldwideBox Office': '$2,845,899,541',
  'DomesticBox Office': '$760,507,625',
  'InternationalBox Office': '$2,085,391,916'},
 {'Rank': '2',
  'Year': '2019',
  'Movie': 'Avengers: Endgame',
  'WorldwideBox Office': '$2,797,800,564',
  'DomesticBox Office': '$858,373,000',
  'InternationalBox Office': '$1,939,427,564'},
 {'Rank': '3',
  'Year': '1997',
  'Movie': 'Titanic',
  'WorldwideBox Office': '$2,207,986,545',
  'DomesticBox Office': '$659,363,944',
  'InternationalBox Office': '$1,548,622,601'}]

In [4]:
type(movies_data)

list

In [5]:
type(movies_data[0])

dict

In [6]:
for record in movies_data:
    print(f"The movie {record['Movie']}, grossed {record['WorldwideBox Office']} in {record['Year']}")

The movie Avatar, grossed $2,845,899,541 in 2009
The movie Avengers: Endgame, grossed $2,797,800,564 in 2019
The movie Titanic, grossed $2,207,986,545 in 1997
The movie Star Wars Ep. VII: The Force Awakens, grossed $2,064,615,817 in 2015
The movie Avengers: Infinity War, grossed $2,044,540,523 in 2018
The movie Jurassic World, grossed $1,669,979,967 in 2015
The movie The Lion King, grossed $1,654,367,425 in 2019
The movie Furious 7, grossed $1,516,881,526 in 2015
The movie The Avengers, grossed $1,515,100,211 in 2012
The movie Frozen II, grossed $1,446,925,396 in 2019


### JSON Pros and Cons

* Pros:
    * Very well supported in modern languages, technologies and infrastructures
    * Can be used as the basis for more performance-optimized formats Parquet or Avro (discussed next)
    * Supports hierarchical structures abstracting the need for complex relationships
    * The *defacto* standard in NoSQL databases
* Cons:
    * Much smaller footprint than XML but still fairly large due to repeated field names
    * Not easy to index
    * Some tentatives to add a schema but not commonly used

### AVRO File Format

* AVRO is an advanced form of the JSON format
  * Leverages some of the advantages of JSON while mitigating some of its disadvantages
* Stores both a file definition (a schema itself written in JSON) and a binary format
  * Said to be self-descriptive because it can include the schema and documentation in the header of the file containing the data
* Serializes the data in a compact binary format
  
* Released by the Hadoop working group in 2009 to use with Hadoop Systems
* It is a row-based format that has a high degree of splitting
* Provides mechanism to manage schema evolution
* support for most modern languages, including Python via the `avro` library

In [ ]:
### AVRO Schema
{
  "type": "record",
  "name": "Student",
  "namespace": "hawaii.edu",
  "fields": [
    {
      "name": "first_name",
      "type": "string"
    },
    {
      "name": "last_name",
      "type": "string"
    },
    {
      "name": "major",
      "type": "string"
    },
    {
      "name": "DOB",
      "type": "string"
    },
    {
      "name": "student_id",
      "type": "string"
    }
  ]
}



### AVRO Data

* For fixed-length types like integers, floats, or fixed-length arrays, we know the exact number of bytes to read. 
 * For instance, a 32-bit integer will always be 4 bytes.
 * In text numbers or other data types are often represented as human-readable text
   *  4162554 takes seven bytes in a text-based format like JSON (4, 1, 6, 2, 5, 5, 4).
*  For variable-length data types like strings, Avro typically uses a length-prefix encoding.
   * length of the data is written first, followed by the actual data.
   * This allows Avro to know exactly how many bytes to read for each value.
     * Length: 2, Data: H, I 
*  For complex types like arrays or records, Avro uses the schema to determine the structure and types of nested values.
*  Does not need to use special characters for delimiters, the encoding is more compact.





### Pros and Cons

* Pros:
    * Binary data minimizes file size and maximizes efficiency
    * A reliable support for schema evolution
      * Supports new, missing, or changed fields.
      * This allows old software to read new data, and new software to read old data
      * It is a critical feature if your data can change.
* Cons:
    * Data is not human readable
    * all the cons of a row-based format

### Analytical optimized Formats

- In analytical queries, a  subset of all the available columns is often needed. 
  - With columnar storage, the system can read only the data of interest, skipping over irrelevant columns, which is much faster.
    - Less data to read
- With columnar formats, metadata about each chunk of data can be stored, allowing the query engine to skip over chunks that are irrelevant to the query, making data retrieval much faster.
- Modern databases and data processing frameworks often use a technique called vectorized query execution. In this approach, operations are performed on an entire column at a time instead of row-by-row, exploiting modern CPU architecture for data-level parallelism.


Query Performance:



### Introduction to Parquet

* Parquet is a specialized file format designed for efficient storage and querying of large datasets.
* It's fast for read operations
    * Reading a specific column doesn't require scanning the entire file.
    * Column-based storage enables faster query performance due to vectorized reading.
* Similar data in columns allows for efficient compression algorithms.
* allows schema evolution.
* Initially developed by Twitter and Cloudera.
* Now an open-source project under Apache Software Foundation.
* Especially useful for data with a large number of columns.

### Anatomy of a Parquet File
* File Metadata: The starting point of the file, contains metadata like schema.
* Column Chunks: Data is organized into sets of columns, each set called a chunk.
* Row Groups: Each chunk can be split further into row groups to optimize reads.
* Metadata: Contains file statistics and optional key-value pairs for custom metadata.




### Data Context
* 
Suppose you have a simplified dataset with the following columns:

  * Transaction_ID (e.g., 001, 002, ...)
  * Time (e.g., 10:01, 10:02, ...)
  * Number_of_Items_Purchased (e.g., 4, 2, ...)
  * Transaction_Total (e.g., 20.50, 10.00, ...)


### File Header

* The File Header provides the Parquet version and schema information. 
  * It essentially lays out what kinds of columns (and their data types) the reader should expect. 
```
message transaction_schema {
  required int64 Transaction_ID;
  required binary Time (UTF8);
  required int32 Number_of_Items_Purchased;
  required double Transaction_Total;
  required binary City (UTF8);
}
```

### Column Chunks
* Column Chunks are contiguous blocks of data from a single column. 
* In our example, one Column Chunk might store a million Transaction_ID values
* another might store a million Time values, and so on.
```
Column Chunk for Transaction_ID: [T001, T002, ... (up to 1,000,000  entries)]
Column Chunk for Time: [10:01, 10:02, ... (up to 1,000,000  entries)]
Column Chunk for Number_of_Items_Purchased: [1, 2, ... (up to 1,000,000  entries)]
Column Chunk for Transaction_Total: [20.50, 10.00, ... (up to 1,000,000  entries)]
Column Chunk for City: [Honolulu, Montreal, ... (up to 1,000,000  entries)]
```

### Row Groups

* Row groups bundle together a subset of Column Chunks to make reads more efficient. 
  * Each Row Group includes a set of Column Chunks, 
    * i.e., column for a subset of rows. 

* ex. Row Group 1 might contain the first million records for each column
```
Row Group 1:
  - Column Chunk 1 for Time: [10:01, 10:02, ... (up to 1,000,000 entries)]
  - Column Chunk 1 for Number_of_Items_Purchased: [1, 2, ... (up to 1,000,000 entries)]
  - Column Chunk 1 for Transaction_Total: [20.50, 10.00, ... (up to 1,000,000 entries)]
  - Column Chunk 1 for City: [Honoluly, montreal, ... (up to 1,000,000 entries)]
```

### Metadata
* Contains information that helps in reading and interpreting the data effectively. 

* This can include:
  * Summary statistics (e.g., min and max values for each column)
  * Compression type used for each column
  * Optional custom key-value pairs, ex. `{ "regions": "north-america", "requested-by": "john"}`
  * Schema definitions.
  

### Data Partitioning in Parquet

* Basic Partitioning: Data can be divided into sub-folders based on a column value.
* Nested Partitions: You can create partitions within partitions based on multiple columns.
* Example Folder Structure, where we're  split on the similar values of the MONTH  or department

```python
/root_directory  
    /HOUR=10
        CITY=HONOLULU
           data..
        CITY=MONTREAL
            data..
        
    /HOUR=11
        CITY=HONOLULU
           data..
        CITY=MONTREAL
            data..

    ...      
```

### Analytics and Reading Data

* If you've partitioned your data well, you generally will not have to read all the files.
  * Read the partitions that are relevant to your query. 
  * For instance, if you're interested in transactions that occurred in the 10:00 hour, you would only read the files under the /Hour=10 directory.

* High-level tools that can manage partitions and fetch only relevant data when using Parquet include:

 * Apache Spark (covered later in class), Presto (open-source distributed SQL) Amazon Athena (based on Presto and optimized for Amazon S3), 
 * Many other open source or commercial high-level tools and query engines


### Exmaple: python


```python
from pyspark.sql import SparkSession

# Initialize a SparkSession
spark = SparkSession.builder \
    .appName("Read Parquet Example") \
    .getOrCreate()

# Read all data from a Parquet file
df = spark.read.parquet("/root_directory/")

# Filter to only include data where Hour equals 10
df_filtered = df.filter(df.Hour == 10)

# Optionally, you can show the filtered DataFrame
df_filtered.show() ,.
```



In [ ]:
### Exmaple: Athena or Presto (SQL)
```SQL
SELECT * FROM "database"."table" WHERE Hour = '10';
```

### PARQUET PROS and CONS

* Pros: 
 * Highly compressible since data is stored column-wise (compression rates up to 75%)
   * Can use different compression algorithms with different datatypes
 * Seamless splittability across columns.
 * Optimized for reading data and ideal for read-intensive tasks
   * Can use parallelization to read different columns.

* Cons:
 * Very slow at writing data and not good with write-intensive applications
 * Does not support updates on the data as Parquet files are immutable.


### Memory Mapping for Working with Large Files

* Memory Mapping is a technique to map either a segment or an entire file from disk into virtual memory.
  * Links a file on disk directly to a section of virtual memory.
* Only the necessary portions of a file are loaded into RAM, as and when needed making efficient use of memory.
* Allows programs to work with files larger than available physical RAM.


### Reading a File From Disk

1. An application initiates a read() system call, moving the request from user space to kernel space.
2. The kernel instructs the hardware to fetch the needed data from the disk.
3. Data is loaded into a kernel buffer, often via Direct Memory Access (DMA) to bypass CPU involvement.
4. Kernel copies data to a user-space buffer specified in the read() call.
  * Kernel Buffers: Optimized for system tasks, reside in privileged memory.
  * User-Space Buffers: For application tasks, accessible by the application.
5. Application proceeds with its logic in user space, using the data in its buffer.
6. A write() system call copies data from user space to a kernel socket buffer.
7. Kernel writes this data to the hardware.
8. Hardware confirms the completion of the write operation to the kernel.
9. write() system call returns, completing the write operation.

### READ FILE

![](https://www.dropbox.com/s/0uaxprebndgw1sn/read_file.png?dl=1)

### Memory Mapping a File From Disk

* `mmap()`: A System Call for Memory Mapping
  * This system call requests the mapping of a file into the application's address space.
  * The request transitions from user space to kernel space to establish the mapping.
* The kernel communicates with the hardware to map the relevant file data into a memory section (RAM).
  * This is typically done "lazily," meaning data may not be loaded until accessed.
* Once the mapping is set up, the application can directly access and modify the data as if it were in its own memory space.
* If data in the mapped memory region is altered, the kernel will eventually flush these changes back to the hardware.
  * The operating system optimizes the timing of this flush based on various factors like system load and other I/O operations.
* `munmap()`:  This system call is used to unmap a previously mapped memory section.
  * It signals the kernel to remove the mapping and release the resources.

### MMAP
![](https://www.dropbox.com/s/eezmaerp24s45s8/mmap.png?dl=1)

### The `time` Command: Overview

The OS tracks time spent executing a program, broken down into CPU times and Wall time.

* CPU Times: The CPU's work time on your task.
  * User Time: Time spent executing your code.
  * System Time: Time on system tasks like memory allocation.
  * Total Time: Sum of User and System Time.
* Wall Time: Real-world elapsed time for your command.

* Important Notes:
  * Wall Time can be < CPU Time due to multitasking.
  * Wall Time can be > CPU Time because of "idle time" (e.g., waiting for disk reads or network data).

### The `time` Command: Refresher

* In a Jupyter Notebook, %time and %%time are magic commands that help you measure the execution time of code, but they are used in slightly different contexts:


* `%time` is used for timing a single line of code. 
```python
%time x = sum(range(100))
```
* `%%time` is used to time the entire cell. 
```python
# Meaures the complete block
%%time
x = sum(range(100))
y = sum(range(200))
```

In [5]:
#!wget -P https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2022-06.parquet

In [2]:
%%time
import pandas as pd
data = pd.read_parquet("~/Downloads/fhvhv_tripdata_2022-06.parquet")
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17780075 entries, 0 to 17780074
Data columns (total 24 columns):
 #   Column                Dtype         
---  ------                -----         
 0   hvfhs_license_num     object        
 1   dispatching_base_num  object        
 2   originating_base_num  object        
 3   request_datetime      datetime64[ns]
 4   on_scene_datetime     datetime64[ns]
 5   pickup_datetime       datetime64[ns]
 6   dropoff_datetime      datetime64[ns]
 7   PULocationID          int64         
 8   DOLocationID          int64         
 9   trip_miles            float64       
 10  trip_time             int64         
 11  base_passenger_fare   float64       
 12  tolls                 float64       
 13  bcf                   float64       
 14  sales_tax             float64       
 15  congestion_surcharge  float64       
 16  airport_fee           float64       
 17  tips                  float64       
 18  driver_pay            float64       
 19

In [4]:
# %%time
# data.to_csv("~/Downloads/fhvhv_tripdata_2022-06.csv")

CPU times: user 1min 58s, sys: 3.6 s, total: 2min 2s
Wall time: 2min 3s


In [9]:
import psutil
import os

def print_mem():
    gig = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 3
    print(f"{gig} gigabytes")
print_mem()

4.2209930419921875 gigabytes


In [13]:
%time df = pd.read_csv("~/Downloads/fhvhv_tripdata_2022-06.csv")

CPU times: user 25.2 s, sys: 3.33 s, total: 28.5 s
Wall time: 29.3 s


In [14]:
print_mem()

14.629287719726562 gigabytes


In [15]:
del df

In [29]:
print_mem()

6.766876220703125 gigabytes


In [4]:
import mmap
import os

In [31]:
%%time
file_handle = open("/Users/mahdi/Downloads/fhvhv_tripdata_2022-06.csv", 'r+b') 
mmap_file = mmap.mmap(file_handle.fileno(), 0)


CPU times: user 419 µs, sys: 723 µs, total: 1.14 ms
Wall time: 17.8 ms


In [32]:
print_mem()

6.766632080078125 gigabytes


In [33]:
line = mmap_file.readline()
print(f"Line 0:\t{line}")


Line 0:	b',hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag\n'


In [34]:
line = mmap_file.readline()
print(f"Line 1:\t{line}")


Line 1:	b'0,HV0003,B03404,B03404,2022-06-01 00:15:35,2022-06-01 00:17:20,2022-06-01 00:17:41,2022-06-01 00:25:41,234,114,1.5,480,7.68,0.0,0.23,0.68,2.75,0.0,1.0,9.36,N,N, ,N,N\n'


In [35]:
mmap_file.seek(0)
line = mmap_file.readline()
print(f"Line 0:\t{line}")

Line 0:	b',hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag\n'


In [37]:
mmap_file.seek?

In [36]:
mmap_file.seek(2)
line = mmap_file.readline()
print(f"Starting at 2:\t{line}")

Starting at 2:	b'vfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag\n'


In [38]:
mmap_file.seek(0)
mmap_file.find(b'\n')

353

In [41]:
mmap_file.seek(353+1)
line = mmap_file.readline()
print(f"{line}")


b'0,HV0003,B03404,B03404,2022-06-01 00:15:35,2022-06-01 00:17:20,2022-06-01 00:17:41,2022-06-01 00:25:41,234,114,1.5,480,7.68,0.0,0.23,0.68,2.75,0.0,1.0,9.36,N,N, ,N,N\n'


In [42]:
mmap_file.tell()

520

In [43]:
%%time
data = open("/Users/mahdi/Downloads/fhvhv_tripdata_2022-06.csv", 'r').read()
len(data)

CPU times: user 263 ms, sys: 726 ms, total: 989 ms
Wall time: 988 ms


2979703837

In [48]:
chr(mmap_file[0])

'o'

In [52]:
mmap_file[0:12]


b'location_key'

In [54]:
# almost equivalent to 
[x.to_bytes(1, 'big') for x in mmap_file[0:12]]


[b'l', b'o', b'c', b'a', b't', b'i', b'o', b'n', b'_', b'k', b'e', b'y']

In [56]:
# equivalent to 
b''.join([x.to_bytes(1, 'big') for x in mmap_file[0:12]])

b'location_key'

In [57]:
# almsot equivalent to 
"".join([chr(x) for x in mmap_file[0:12]])

'location_key'

In [7]:
%%time
file_handle = open("/Users/mahdi/Downloads/fhvhv_tripdata_2022-06.csv", 'r+b') 
total_lines = 0 
mmap_file = mmap.mmap(file_handle.fileno(), 0)
while mmap_file.readline():
    total_lines += 1
total_lines        

CPU times: user 1.74 s, sys: 366 ms, total: 2.11 s
Wall time: 2.11 s


17780076

In [77]:
!du -sch ~/Downloads/aggregated_100.csv

101G	/Users/mahdi/Downloads/aggregated_100.csv
101G	total


In [78]:
%%time
file_handle = open("/Users/mahdi/Downloads/aggregated_100.csv", 'r+b') 


CPU times: user 232 µs, sys: 986 µs, total: 1.22 ms
Wall time: 781 µs


In [5]:
%%time
total_lines = 0 
file_handle = open("/Users/mahdi/Downloads/fhvhv_tripdata_2022-06.csv",) 
while file_handle.readline():
    total_lines += 1
total_lines        

CPU times: user 2.46 s, sys: 380 ms, total: 2.84 s
Wall time: 2.85 s


17780076

In [8]:
# The following creates a csv file with randome fields with num_cols and 10M entrie

import csv
import random
import string

num_rows = 10000000  # 10 million rows
num_cols = 100  # 100 columns

# This will generate a random string of length n
def random_string(n=10):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=n))

# Open a new CSV file in write mode ('w')
with open('large_file.csv', 'w', newline='', buffering=10485760) as csvfile:  # 10 MB buffer
    csv_writer = csv.writer(csvfile)
    
    # Write the header
    csv_writer.writerow([f"Column_{i}" for i in range(num_cols)])

    # Write the rows
    for _ in range(num_rows):
        csv_writer.writerow([random_string() for _ in range(num_cols)])


In [18]:
%%time
file_handle = open("large_file.csv", 'r+b') 
total_lines = 0 
mmap_file = mmap.mmap(file_handle.fileno(), 0)
while mmap_file.readline():
    total_lines += 1
total_lines        

CPU times: user 1.88 s, sys: 4.32 s, total: 6.2 s
Wall time: 23.6 s


10000001

In [19]:
%%time
total_lines = 0 
file_handle = open("large_file.csv",) 
while file_handle.readline():
    total_lines += 1
total_lines        

CPU times: user 7.84 s, sys: 1.43 s, total: 9.27 s
Wall time: 9.36 s


10000001

In [30]:
%%time
with open("large_file.csv", 'r+b') as file_handle:
    total_lines = 0
    mmap_file = mmap.mmap(file_handle.fileno(), 0)
    chunk_size = 1 << 17  # 64 KiB
    while True:
        chunk = mmap_file.read(chunk_size)
        if not chunk:
            break
        total_lines += chunk.count(b'\n')


CPU times: user 4.29 s, sys: 1.53 s, total: 5.82 s
Wall time: 5.83 s
